In [20]:
import pandas as pd
import rdflib
from rdflib import URIRef
from owlready2 import get_ontology
from tqdm import tqdm
from glob import glob
import logging
import os
from sklearn.model_selection import train_test_split


In [21]:
dataset_name = 'OWL2DL-1'

**Filter unnecessary inferred triples**

In [22]:
ontology = get_ontology(f'datasets/{dataset_name}.owl').load()
subject_resources = list(ontology.individuals())
named_individuals = [URIRef(x.iri) for x in subject_resources]
print(f'# Subject-Resources: {len(subject_resources)}')

# Subject-Resources: 3668


In [23]:
def filter_triples(inferred_path, filtered_inferred_path):    
    g_inferred = rdflib.Graph()
    g_inferred.parse(inferred_path, format="ttl")
    g_inferred_filtered = rdflib.Graph()
    for triple in g_inferred: 
        if triple[0] in named_individuals or triple[2] in named_individuals:
            g_inferred_filtered.add(triple)
    g_inferred_filtered.serialize(filtered_inferred_path, format="ttl")

In [25]:
for input_graph_path in tqdm(sorted(glob('MyJenaProject/output/' + "*"))):
    filter_triples(input_graph_path, input_graph_path.replace('output','output_filtered'))

100%|██████████| 3662/3662 [3:50:06<00:00,  3.77s/it]  


**Create train, test, val sets**

In [2]:
def get_graph_type(s):
    s = s.split('/')[-1]
    s = s.split('_')[:-1]
    s = "_".join(s)
    return s

In [3]:
def get_files_df(INPUT_GRAPHS_FOLDER, INFERENCE_GRAPHS_FOLDER):
    logging.info("Creating dataframe for OWL2DL-1 input/inference pairs")
    rdf_files = []
    for input_graph_path in tqdm(sorted(glob(INPUT_GRAPHS_FOLDER + "*"))):
        input_graph_file = os.path.basename(input_graph_path)
        inference_path = INFERENCE_GRAPHS_FOLDER + input_graph_file.replace('.ttl','_inferred.ttl')
        graph_type = get_graph_type(input_graph_path)
        rdf_pair = {"input_graph_file": input_graph_path, "inference_file": inference_path, "graph_type": graph_type}
        rdf_files.append(rdf_pair)
    files_df = pd.DataFrame.from_dict(rdf_files)
    return files_df

In [ ]:
files_df = get_files_df('MyJenaProject/input/', 'MyJenaProject/output/')

In [5]:
# Remove classes for which only one instance exists
df_count = pd.DataFrame(files_df['graph_type'].value_counts())
graph_type_2_keep = df_count[df_count['count'] > 1].index
files_df = files_df[files_df['graph_type'].isin(graph_type_2_keep)]

In [6]:
def train_validate_test_split(df, train_percent=0.6, validate_percent=0.2, stratify=None, seed=1):
    val_test_percent = 1 - train_percent
    test_percent = (1 - (train_percent + validate_percent))
    test_percent = test_percent / (test_percent + validate_percent)
    if stratify:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed,
                                                 stratify=df[stratify])
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed,
                                           stratify=df_val_test[stratify])
    else:
        df_train, df_val_test = train_test_split(df, test_size=val_test_percent, random_state=seed)
        df_val, df_test = train_test_split(df_val_test, test_size=test_percent, random_state=seed)
    return df_train, df_val, df_test

In [7]:
rdf_data_train, rdf_data_val, rdf_data_test = train_validate_test_split(files_df,
                                                                        train_percent=0.6,
                                                                        validate_percent=0.2,
                                                                        stratify="graph_type",
                                                                        seed=1)

In [8]:
def write_nt_file(data, output_file, old_iri, new_iri):
    nt_files_orig = data['input_graph_file']
    nt_files_inferred = data['inference_file']
    
    # Concatenate the file lists from the original and inferred graphs
    nt_files = pd.concat([nt_files_orig, nt_files_inferred])

    with open(output_file, 'w') as outfile:
        for nt_file in nt_files:
            with open(nt_file, 'r') as infile:
                for line in infile:
                    # Replace the old IRI with the new IRI in the triple
                    updated_line = line.replace(old_iri, new_iri)
                    outfile.write(updated_line)
                outfile.write("\n")  # Write a newline after each file


In [9]:
write_nt_file(rdf_data_train, 'datasets/bin/OWL2DL-1_train.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_train')
write_nt_file(rdf_data_val, 'datasets/bin/OWL2DL-1_val.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_val')
write_nt_file(rdf_data_test, 'datasets/bin/OWL2DL-1_test.owl', 
              'https://kracr.iiitd.edu.in/OWL2Bench', 'https://kracr.iiitd.edu.in/OWL2Bench_test')